In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np
import seaborn as sns
import time
import difflib
from collections import Counter

In [2]:
import google.auth
from google.cloud import bigquery
import pandas_gbq

credentials, your_project_id = google.auth.default(
    scopes = ['https://www.googleapis.com/auth/cloud-platform']
)

gcp_project = 'GCP_PROJECT_NAME_HERE'

client = bigquery.Client(project=gcp_project)

In [3]:
def run_query():
    '''Runs query and returns a dataframe'''
    
    query = """
    # BQ Query HERE
    ;
    """

    query_job = client.query(
        query, 
        location = 'US'
    )

    df = query_job.result().to_dataframe()
    
    return(df)

In [4]:
def get_pitcher_list(df):
    '''Function takes a dataframe and returns a list of each pitcher id present in the df'''
    
    pitcher_list = df.pitcher.unique().tolist()
    
    return(pitcher_list)

In [5]:
def get_sequence_list(pitcher_id, df):
    '''Function takes a pitcher id and dataframe; returns a list of lists (each at bat sequence is a list within the master list)'''
    
    # filter dataframe for pitcher
    df_filtered = df.loc[df['pitcher'] == pitcher_id]
    
    # turn striny value from sequence2 field into a list
    df_filtered['sequence2_lst'] = df_filtered.sequence2.apply(lambda x: x.split(','))
    
    # take all row values of sequence2_lst and turn into a list of lists
    seq_list = df_filtered['sequence2_lst'].values.tolist()
    
    # also get name of pitcher
    try:
        player_name = df_filtered['player_name'].iloc[0]
    except IndexError:
        player_name = 'Unknown'
    
    
    return(seq_list, player_name)

In [6]:
def get_sequence_matcher_ratio_average(pitcher_id, player_name, sequence_list):
    '''Function takes a pitcher id, player name, and list of sequence lists; returns a dictionary with the player's information and average similarity ratio'''
    
    player_results = []
    
    loop_index = 0
    total_match_len = 0
    total_len = 0

    for x in range (len(sequence_list)):

        loop_index = 0
        for loop_index in range(len(sequence_list)):
            if x == loop_index:
                pass
            else:
                sm = difflib.SequenceMatcher(None, sequence_list[x], sequence_list[loop_index])
                
                matches = sum(match_len[-1] for match_len in sm.get_matching_blocks())
                total_match_len += matches
                
                seq_len = len(sequence_list[x]) + len(sequence_list[loop_index])
                total_len += seq_len
                
                loop_index +=1

    try:
        overall_ratio = 2* (total_match_len / total_len)
    except ZeroDivisionError:
        overall_ratio = 0
    
    # results
    results_dict = {'Pitcher':pitcher_id, 'Player_Name': player_name, 'Overall_Ratio': overall_ratio, 'Match' : total_match_len, 'Len':total_len}
    
    return(results_dict)

In [7]:
def generate_pitcher_results(pitcher_list, df):
    '''Function takes pitcher list and dataframe, runs a loop of each pitcher, returns their results dictionary, and then appends to a master dataframe which is returned by the function'''
    
    results_list = []

    
    for p in pitcher_list:
        sl, pn  = get_sequence_list(p, df)

        rd = get_sequence_matcher_ratio_average(p, pn, sl)

        results_list.append(rd)
        
    results_df = pd.DataFrame(results_list)  

    return(results_df)

In [8]:
def run_full_program():
    '''Function executes the entire program'''
    
    df = run_query()

    pitcher_list = get_pitcher_list(df)

    df_results = generate_pitcher_results(pitcher_list, df)
    
    return(df_results)

In [9]:
start = time.time()

file = run_full_program()

print('Runtime: ', time.time() - start)

Runtime:  2.3376150131225586


In [10]:
file

,Pitcher,Player_Name,Overall_Ratio,Match,Len
0,605483,"Snell, Blake",0.426535,1077,5050
